## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Rikitea,PF,-23.1203,-134.9692,78.22,74,100,17.02
1,1,Avarua,CK,-21.2078,-159.7750,82.45,78,20,19.57
2,2,Bluff,NZ,-46.6000,168.3333,52.81,84,100,5.46
3,3,East London,ZA,-33.0153,27.9116,67.14,82,0,10.36
4,4,Puerto Ayora,EC,-0.7393,-90.3518,75.54,93,7,4.00


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 100


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Rikitea,PF,-23.1203,-134.9692,78.22,74,100,17.02
1,1,Avarua,CK,-21.2078,-159.7750,82.45,78,20,19.57
3,3,East London,ZA,-33.0153,27.9116,67.14,82,0,10.36
4,4,Puerto Ayora,EC,-0.7393,-90.3518,75.54,93,7,4.00
5,5,Morvi,IN,22.8167,70.8333,63.61,32,0,4.79
7,7,Castro,BR,-24.7911,-50.0119,64.17,98,100,4.05
8,8,Innisfail,AU,-17.5333,146.0333,79.48,91,100,1.14
9,9,Micheweni,TZ,-4.9667,39.8333,80.19,79,99,9.08
11,11,Padang,ID,-0.9492,100.3543,84.25,72,84,3.91
13,13,Hilo,US,19.7297,-155.0900,74.93,81,100,9.22


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.shape[0] == 0
len(preferred_cities_df) == 0
len(preferred_cities_df.index) == 0
preferred_cities_df.empty

False

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)



,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Rikitea,PF,78.22,-23.1203,-134.9692,
1,Avarua,CK,82.45,-21.2078,-159.7750,
3,East London,ZA,67.14,-33.0153,27.9116,
4,Puerto Ayora,EC,75.54,-0.7393,-90.3518,
5,Morvi,IN,63.61,22.8167,70.8333,
7,Castro,BR,64.17,-24.7911,-50.0119,
8,Innisfail,AU,79.48,-17.5333,146.0333,
9,Micheweni,TZ,80.19,-4.9667,39.8333,
11,Padang,ID,84.25,-0.9492,100.3543,
13,Hilo,US,74.93,19.7297,-155.0900,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
   "radius": 5000,
   "type": "lodging",
   "key": g_key
}
# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
try:
    hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
except (IndexError):
    print("Hotel not found... skipping.")

In [14]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [11]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [17]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template  = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [16]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))